!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
pip install --no-cache-dir --index-url https://pypi.nvidia.com pytorch-quantization !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
 ./ch transform --config configs/examples/toy_uniform_tensorRT.toml --load /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt --load-type pl
./ch train --config configs/examples/toy_uniform_tensorRT.toml
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

sudo apt update && sudo apt upgrade -y 

wget https://repo.continuum.io/archive/Anaconda3-2023.09-0-Linux-x86_64.sh 

bash Anaconda3-2023.09-0-Linux-x86_64.sh 

Close and reopen terminal

source /root/.bashrc

conda config --set auto_activate_base false

git clone https://github.com/mau-mar/mase/

cd mase

bash scripts/init-conda.sh

source /opt/conda/etc/profile.d/conda.sh

conda activate mase

conda config --add channels conda-forge

git checkout origin/mauro-tensorRT-integration

cuda-python
absl-py
scipy
prettytable
sphinx-glpi-theme

./ch train --config configs/examples/toy_uniform_tensorRT.toml

In [26]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

# # figure out the correct path
machop_path = Path(".").resolve().parent /"machop"
assert machop_path.exists(), "Failed to find machop at: {}".format(machop_path)
sys.path.append(str(machop_path))
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch_tensorrt

from torch.utils.tensorboard import SummaryWriter

import pytorch_quantization
from pytorch_quantization import nn as quant_nn
from pytorch_quantization import quant_modules
from pytorch_quantization.tensor_quant import QuantDescriptor
from pytorch_quantization import calib
from tqdm import tqdm

print(pytorch_quantization.__version__)

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
    tensorrt_quantize_transform_pass,
    tensorrt_calibrate_transform_pass,
    tensorrt_train_transform_pass,
    )
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

INFO     Set logging level to info
I0304 15:13:35.392863 140720156907328 logger.py:44] Set logging level to info


2.1.3


In [27]:
model_name = "jsc-tiny"
dataset_name = "jsc"
max_epochs = 1
batch_size = 256
learning_rate = 1e-3
accelerator = "gpu"

data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)
data_module.prepare_data()
data_module.setup()

In [28]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt"
model_info = get_model_info(model_name)
# quant_modules.initialize()
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)

INFO     Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt
I0304 15:13:38.631031 140720156907328 checkpoint_load.py:85] Loaded pytorch lightning checkpoint from /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt


In [29]:
mg = MaseGraph(model=model)

TraceError: symbolically traced variables cannot be used as inputs to control flow

In [ ]:
import toml

# Path to your TOML file
toml_file_path = '/root/mase/machop/configs/examples/toy_uniform_tensorRT.toml'

# Reading TOML file and converting it into a Python dictionary
with open(toml_file_path, 'r') as toml_file:
    pass_args = toml.load(toml_file)

def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

print(pass_args)
print("\n")
pass_args = flatten_dict(pass_args)
print(pass_args)

{'model': 'jsc-tiny', 'dataset': 'jsc', 'max_epochs': 1, 'batch_size': 256, 'learning_rate': 0.001, 'accelerator': 'gpu', 'passes': {'tensorrt-calibrate': {'by': 'type', 'calibrator': ['percentile', 'max', 'mse', 'entropy'], 'percentiles': [99], 'report': True}, 'tensorrt-quantize': {'default': {'config': {'name': 'integer', 'data_in_width': 8, 'data_in_frac_width': 4, 'weight_width': 8, 'weight_frac_width': 9, 'bias_width': 8, 'bias_frac_width': 9}}}}}


{'model': 'jsc-tiny', 'dataset': 'jsc', 'max_epochs': 1, 'batch_size': 256, 'learning_rate': 0.001, 'accelerator': 'gpu', 'passes_tensorrt-calibrate_by': 'type', 'passes_tensorrt-calibrate_calibrator': ['percentile', 'max', 'mse', 'entropy'], 'passes_tensorrt-calibrate_percentiles': [99], 'passes_tensorrt-calibrate_report': True, 'passes_tensorrt-quantize_default_config_name': 'integer', 'passes_tensorrt-quantize_default_config_data_in_width': 8, 'passes_tensorrt-quantize_default_config_data_in_frac_width': 4, 'passes_tensorrt-quantiz

In [30]:
task = 'classification'

# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task=task,
    which_dataloader="val",
)

train_generator = InputGenerator(
    model_info=model_info,
    data_module=data_module,
    task=task,
    which_dataloader="train",
)

val_generator = InputGenerator(
    model_info=model_info,
    data_module=data_module,
    task=task,
    which_dataloader="train",
)

pass_args["train_generator"] = train_generator
pass_args["val_generator"] = val_generator
pass_args['data_loader'] = data_module.train_dataloader()

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
_ = model(**dummy_in)

mg, _ = init_metadata_analysis_pass(mg, None)
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})
mg, _ = add_software_metadata_analysis_pass(mg, None)

AttributeError: 'Linear' object has no attribute '__name__'

In [14]:
'''  
./ch transform --config configs/examples/toy_uniform_tensorRT.toml --load /root/mase/TensorRTDev/jsc-tiny_classification_jsc_2024-03-03/software/training_ckpts/best.ckpt --load-type pl
'''
# Calibrate model (passing data samples to the quantizer and deciding the best amax for activations)
mg = tensorrt_calibrate_transform_pass(mg, pass_args=pass_args_tensorrt_calibrate)

# Train on fake quantization
mg = tensorrt_train_transform_pass(mg, pass_args=pass_args_tensorrt_calibrate)


# Convert and store to ONNX and then TensorRT
trt_graph = tensorrt_quantize_transform_pass(mg, pass_args=pass_args_tensorrt_calibrate)

# Analysis pass
...

KeyError: 'by'

In [28]:
# for node in mg.fx_graph.nodes:
#     print(node.target)
# print(mg.model)
i = 0
for name, module in mg.model.named_modules():
    print(i,module)
    i += 1

0 GraphModule(
  (feature_layers): Module(
    (0): QuantConv2d(
      3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)
      (_input_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
      (_weight_quantizer): TensorQuantizer(8bit fake per-tensor amax=dynamic calibrator=HistogramCalibrator scale=1.0 quant)
    )
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(128, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(256, eps=1e-05, momentum=0.9, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 25